# Open3d

## 1.介绍
Open3D has been in development since 2015 by Intel. 

So Far, the newest version is 0.17.0.

The Open3D backend is implemented in C++11, is highly optimized, and is set up for OpenMP parallelization. 

使用Python binding,提供前端`Python`接口，后端用`C++11`实现。

MIT license 

Introduction Thesis:[http://www.open3d.org/wordpress/wp-content/paper.pdf](http://www.open3d.org/wordpress/wp-content/paper.pdf)

### 1.1与PCL相比

Unfortunately, after an initial influx of open-
source contributions, PCL became encumbered by bloat and
is now largely dormant. 


Open3D的代码优化做的更充分，运行速度更快。`Open3D`中的`ICP`算法比`PCL`中的要快`25`倍。

### 1.2主要功能

- **法向量评估**
- **ICP点云拼接**
- **Volumetric integration**

In [32]:
import open3d as o3d
import numpy as np

## 文件读取

Dataset Download URLS: [http://graphics.stanford.edu/data/3Dscanrep/](http://graphics.stanford.edu/data/3Dscanrep/)

Point Cloud/Mesh/Model

![](mesh_model_pc.png)

### Open3d支持的`Mesh`文件类型
![image.png](mesh_format_o3d.png)
### Open3d支持的点云类型

![image.png](pcd_format.png)

In [86]:
filename = "dragon_recon/dragon_vrip_res2.ply"
dragon_mesh = o3d.io.read_triangle_mesh(filename)
print(dragon_mesh)
dragon_mesh.compute_vertex_normals()
print(np.asarray(dragon_mesh.triangles).shape)
print(np.asarray(dragon_mesh.vertices).shape)
o3d.visualization.draw_geometries([dragon_mesh])    # 可视化Mesh

TriangleMesh with 100250 points and 202520 triangles.
(202520, 3)
(100250, 3)


## 保存及可视化点云数据

- **保存点云**： `write_point_cloud(filename, pointcloud, write_ascii=False, compressed=False, print_progress=False)`
  Function to write PointCloud to file
- **可视化点云**：
    - 1. `draw_geometries(geometry_list, window_name='Open3D', width=1920, height=1080, left=50, top=50, point_show_normal=False, mesh_show_wireframe=False, mesh_show_back_face=False)` Function to draw a list of geometry.Geometry objects。
使用`OpenGL`进行渲染。
    - 2.`draw_geometries_with_key_callbacks`参考自[callback](http://whitewell.sakura.ne.jp/Open3D/CustomizedVisualization.html)

In [99]:
dragon_pc = dragon_mesh.sample_points_uniformly(number_of_points=20000)
save_file = "dragon_recon/dragon_vrip_res2.pcd"
o3d.io.write_point_cloud(save_file, dragon_pc)
o3d.visualization.draw_geometries([dragon_pc])    # 可视化点云 +/-可调点云的大小

def rotate_callback(vis):
    ctr = vis.get_view_control()
    ctr.rotate(10.0, 0.0)
    return False
key_to_callback = dict()
key_to_callback[ord("r")] = rotate_callback
o3d.visualization.draw_geometries_with_key_callbacks([dragon_pc], key_to_callback)

## Voxel降采样

刚刚随机采样生成点云时，number_of_points设置的是`20000`,点太多了的时候，可以使用体素`Voxel`来降采样。

`voxel_down_sample(self, voxel_size)`
    Function to downsample input pointcloud into output pointcloud with a voxel. Normals and colors are averaged if they exist.
- **o3d.visualization.read_selection_polygon_volume** 通过空间多边形和最大最小距离来裁剪点云数据
- **compute_convex_hull(...)** method of open3d.cpu.pybind.geometry.PointCloud instance,计算点云的凸包
- **estimate_normals** 计算点云的法向量

In [91]:
max_bound = dragon_pc.get_max_bound()
min_bound = dragon_pc.get_min_bound()
dx, dy, dz = max_bound - min_bound
voxel_size = min([dx, dy, dz]) / 20
print(dragon_pc)
dragon_pc_downsampled = dragon_pc.voxel_down_sample(voxel_size=voxel_size)
print(dragon_pc_downsampled.points)

dragon_pc_downsampled.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=32)) 
o3d.visualization.draw_geometries([dragon_pc_downsampled], point_show_normal=True)

dragon_normals = dragon_pc_downsampled.normals
print(dragon_normals[0])

PointCloud with 20000 points.
std::vector<Eigen::Vector3d> with 4179 elements.
Use numpy.asarray() to access data.
[-0.39450818  0.35314528  0.84832288]


In [66]:
demo_crop_data = o3d.data.DemoCropPointCloud()
pcd = o3d.io.read_point_cloud(demo_crop_data.point_cloud_path)
vol = o3d.visualization.read_selection_polygon_volume(demo_crop_data.cropped_json_path)
chair = vol.crop_point_cloud(pcd)
o3d.visualization.draw_geometries([chair],
                                  zoom=0.7,
                                  front=[0.5439, -0.2333, -0.8060],
                                  lookat=[2.4615, 2.1331, 1.338],
                                  up=[-0.1781, -0.9708, 0.1608])

[Open3D INFO] Downloading https://github.com/isl-org/open3d_downloads/releases/download/20220201-data/DemoCropPointCloud.zip
[Open3D INFO] Downloaded to /home/lx/open3d_data/download/DemoCropPointCloud/DemoCropPointCloud.zip
[Open3D INFO] Created directory /home/lx/open3d_data/extract/DemoCropPointCloud.
[Open3D INFO] Extracting /home/lx/open3d_data/download/DemoCropPointCloud/DemoCropPointCloud.zip.
[Open3D INFO] Extracted to /home/lx/open3d_data/extract/DemoCropPointCloud.


In [84]:
import json
import pprint
pp = pprint.PrettyPrinter(2)

with open(demo_crop_data.cropped_json_path) as f:
    d = json.load(f)
    pp.pprint(d)

bounding_polygon = np.array(d["bounding_polygon"])
bounding_polygon_data = o3d.geometry.PointCloud()
bounding_polygon_data.points = o3d.utility.Vector3dVector(bounding_polygon)
bounding_polygon_data.paint_uniform_color([1, 0.6, 0])
chair_dat = chair.random_down_sample(0.6)
hull, _ = chair_dat.compute_convex_hull()
hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
hull_ls.paint_uniform_color((1, 0, 0))
o3d.visualization.draw_geometries([bounding_polygon_data, hull_ls, chair_dat, o3d.geometry.TriangleMesh.create_coordinate_frame()])


{ 'axis_max': 4.022921085357666,
  'axis_min': -0.763413667678833,
  'bounding_polygon': [ [2.6509309513852526, 0.0, 1.6834473132326844],
                        [2.578642824691715, 0.0, 1.6892074266735244],
                        [2.4625790337552154, 0.0, 1.6665777078297999],
                        [2.2228544982251655, 0.0, 1.6168160446813649],
                        [2.166993206001413, 0.0, 1.6115495157201662],
                        [2.1167895865303286, 0.0, 1.6257706054969348],
                        [2.0634657721747383, 0.0, 1.623021658624539],
                        [2.0568612343437236, 0.0, 1.5853892911207643],
                        [2.1605399001237027, 0.0, 0.9622899325508302],
                        [2.1956669387205228, 0.0, 0.9557274604978507],
                        [2.2191318790575583, 0.0, 0.8873444998210875],
                        [2.248488184792592, 0.0, 0.8704280726701363],
                        [2.6891234157295827, 0.0, 0.941406779889676],
               

## 点云包围框

![](bounding_box3d.png)

有点`[0, 0, 0], [0, 1, 1], [1, 1, 0], [0, 0, 1]`

- **1.get_axis_aligned_bounding_box** 如上图，红色部分包围框就是此函数求得的结果，包围矩形的相互平行的四条边都与坐标轴平行，类似`2D`图像中的水平包围框
- **2.get_oriented_bounding_box** 如上图，绿色部分的包围框就是此函数求得的结果，给出的是空间中的最小包围矩形，类似`2D`图像中的旋转包围框


In [67]:
printr_points = np.array([[0, 0, 0], [0, 1, 1], [1, 1, 0], [0, 0, 1]])
four_pc = o3d.geometry.PointCloud()
four_pc.points = o3d.utility.Vector3dVector(four_points)
four_pc.paint_uniform_color([1, 0.2, 0.4])
aabb = four_pc.get_axis_aligned_bounding_box()
aabb.color = (1, 0, 0)
obb = four_pc.get_oriented_bounding_box()
obb.color = (0, 1, 0)
o3d.visualization.draw_geometries([four_pc, aabb, obb, o3d.geometry.TriangleMesh.create_coordinate_frame()])    # 可视化点云


## 点云与numpy数组转换

In [39]:
# 点云转成numpy数组
print(np.asarray(dragon_pc_downsampled.points).shape) # std::vector<Eigen::Vector3d> to numpy array
# numpy 数组构建点云
random_pcd_points = np.random.randint(0, 255, (2000, 3))
random_pcd = o3d.geometry.PointCloud()
random_pcd.points = o3d.utility.Vector3dVector(random_pcd_points)
random_pcd.paint_uniform_color([1, 0.7, 0])
o3d.visualization.draw_geometries([random_pcd], width=1920, height=1080, left=50, top=50)

(4167, 3)


## 裁剪Mesh


In [63]:
filename = "dragon_recon/dragon_vrip_res2.ply"
dragon_mesh = o3d.io.read_triangle_mesh(filename)
dragon_mesh.triangles = o3d.utility.Vector3iVector(
    np.asarray(dragon_mesh.triangles)[:len(dragon_mesh.triangles) // 2, :])
dragon_mesh.triangle_normals = o3d.utility.Vector3dVector(
    np.asarray(dragon_mesh.triangle_normals)[:len(dragon_mesh.triangle_normals) // 2, :])
dragon_mesh.paint_uniform_color([1, 0.7,0])
print(dragon_mesh.triangles)
o3d.visualization.draw_geometries([dragon_mesh])

std::vector<Eigen::Vector3i> with 101260 elements.
Use numpy.asarray() to access data.
